In [74]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


# Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
# 체인을 수동으로 구현해야 합니다.
# 체인에 ConversationBufferMemory를 부여합니다.
# 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
# 체인에 다음 질문을 합니다:
    # Is Aaronson guilty?
    # What message did he write in the table?
    # Who is Julia?


llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferMemory(return_messages=True)

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriver = vectorstore.as_retriever()


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


invoke_chain("Is Aaronson guilty?")



content="According to the context, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, but he also acknowledges that he has never seen the photograph that disproved their guilt and that it had never existed. Therefore, he concludes that he invented it. This suggests that Aaronson is not actually guilty, but rather a victim of the Party's manipulation."


In [75]:
invoke_chain("What message did he write in the table?")


content='Winston traced "2+2=5" in the dust on the table.'


In [76]:
invoke_chain("Who is Julia?")


content='Julia is a character with whom Winston has a romantic relationship. She is someone he loves deeply, and their relationship is significant in the context of their rebellion against the oppressive regime of the Party.'


In [77]:
print(load_memory(None))
invoke_chain("Is Aaronson guilty?")


[HumanMessage(content='Is Aaronson guilty?'), AIMessage(content="According to the context, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, but he also acknowledges that he has never seen the photograph that disproved their guilt and that it had never existed. Therefore, he concludes that he invented it. This suggests that Aaronson is not actually guilty, but rather a victim of the Party's manipulation."), HumanMessage(content='What message did he write in the table?'), AIMessage(content='Winston traced "2+2=5" in the dust on the table.'), HumanMessage(content='Who is Julia?'), AIMessage(content='Julia is a character with whom Winston has a romantic relationship. She is someone he loves deeply, and their relationship is significant in the context of their rebellion against the oppressive regime of the Party.')]
content="According to the context, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are c